In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

from tfa.losses import SigmoidFocalCrossEntropy

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import PiecewiseConstantDecay
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import os

In [ ]:
data_dir = ""
test_data_dir = ""

In [ ]:
!dir

In [ ]:
IMAGE_SIZE = (224,224)
BATCH_SIZE = 32
seed = np.random.seed(42)

In [ ]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20, 
    data_format='channels_last')

dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
    interpolation="bilinear", seed=seed)


valid_datagen = ImageDataGenerator(
    **datagen_kwargs)

valid_generator = valid_datagen.flow_from_directory(
    data_dir, subset="validation", shuffle=False, **dataflow_kwargs)


train_datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.05, 
    height_shift_range=0.2,
    **datagen_kwargs)

train_generator = train_datagen.flow_from_directory(
    data_dir, subset="training", shuffle=True, **dataflow_kwargs)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255, 
    data_format='channels_last')

test_generator = test_datagen.flow_from_directory(
    test_data_dir, shuffle=False, target_size=IMAGE_SIZE, batch_size=13,
    interpolation="bilinear", seed=seed)

In [ ]:
# VGG16 Architecture
model = Sequential()
model.add(keras.applications.vgg16.VGG16(
                        include_top=False, 
                        weights='imagenet', 
                        input_tensor=None, 
                        input_shape=(224, 224, 3), 
#                         pooling='max',
                        pooling='avg'))

model.add(layers.Flatten(name='flatten'))
model.add(layers.Dense(4096, 
                        activation='relu', 
                        name='fc1'))
model.add(layers.Dense(4096, 
                        activation='relu', 
                        name='fc2'))
model.add(layers.Dense(4, 
                        activation='softmax', 
                        name='predictions'))
model.build()
model.summary()

In [ ]:
# loss = CategoricalCrossentropy(
#     from_logits=False, 
#     label_smoothing=0, 
#     name='categorical_crossentropy')
loss = SigmoidFocalCrossEntropy(
    alpha=0.25,
    gamma=2.0,
    name='sigmoid_focal_cross_entropy')

In [ ]:
optimizer = SGD(
    momentum=0.9, 
    nesterov=True, 
    name='SGD',
)

In [ ]:
model.compile(    
    optimizer=optimizer,
    loss= loss,
    metrics=['accuracy'])        

In [ ]:
callbacks = []

In [ ]:
# def scheduler(epoch):
#     if epoch == 1:
#         return 0.001
#     elif epoch == 2:
#         return 0.0001
#     else:
#         return 0.0001 * np.exp(0.1 * (2 - epoch))
# learning_rate_scheduler_callback = LearningRateScheduler(scheduler)
scheduler = PiecewiseConstantDecay(
    boundaries=[150], 
    values=[0.001, 0.0001]
)
learning_rate_scheduler_callback = LearningRateScheduler(scheduler)
callbacks.append(learning_rate_scheduler_callback)

In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    os.path.join('models','output_models','weights_only_bla_bla_bla_{epoch:02d}_{val_loss:.4f}.hdf5'), 
#     monitor='val_loss', 
    verbose=0, 
    save_best_only=False,
    save_weights_only=True, 
    mode='auto', 
    save_freq='epoch'
)
callbacks.append(model_checkpoint_callback)

In [ ]:
# class_weight = class_weight.compute_class_weight(
#     'balanced', 
#     np.unique(train_generator.classes), 
#     train_generator.classes)
# class_weight = dict(zip(dict(valid_generator.class_indices).values(), class_weight))

In [ ]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size
hist = model.fit(
    train_generator,
    epochs=1, 
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps,
#     class_weight=class_weight,
    callbacks=callbacks
).history

In [ ]:
steps = test_generator.n//test_generator.batch_size
Y_pred = model.predict(test_generator, steps=steps)

In [ ]:
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

In [ ]:
plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(hist["accuracy"])
plt.plot(hist["val_accuracy"])